In [1]:
import torch
import torch.nn as nn

https://github.com/LKLQQ/ViG/blob/dea9ad27c2e5514ec85c3c2a082267d5d427e1e3/src/models/vig/gcn_lib.py#L32
https://github.com/LKLQQ/ViG/blob/dea9ad27c2e5514ec85c3c2a082267d5d427e1e3/src/models/vig/vig.py

https://github.com/lightaime/deep_gcns_torch/blob/master/gcn_lib/dense/torch_edge.py
https://github.com/lightaime/deep_gcns_torch/blob/751382aa2d25e25a2792c133cc99f8cfddae0657/gcn_lib/dense/torch_nn.py#L75

In [3]:
image = torch.randn(1,3,224,224)

In [4]:
H = W = 224
C = 3
P = 16
N=H*W//(P**2)

emb_dim = 768

### Patches

In [8]:
conv_proy = nn.Conv2d(C, emb_dim, kernel_size=P, stride=P)

print(conv_proy(image).shape)

batch = conv_proy(image).flatten(2).transpose(1,2)

torch.Size([1, 768, 14, 14])


In [10]:
batch.shape

torch.Size([1, 196, 768])

### KNN pairwise distance

In [52]:
def pairwise_distance(x):
    """
    Compute pairwise distance of a point cloud.
    Args:
        x: tensor (batch_size, num_points, num_dims)
    Returns:
        pairwise distance: (batch_size, num_points, num_points)
    """
    x_inner = -2*torch.matmul(x, x.transpose(2, 1))
    x_square = torch.sum(torch.mul(x, x), dim=-1, keepdim=True)
    return torch.sqrt(x_square + x_inner + x_square.transpose(2, 1)) #SQRT((x - x.T)^2 = X^2 -2XX.T + X.T^2)

In [115]:
def pairwise_distance(x):
    """
    Compute pairwise distance of a point cloud.
    Args:
        x: tensor (batch_size, num_points, num_dims)
    Returns:
        pairwise distance: (batch_size, num_points, num_points)
    """
    x_inner = -2*torch.matmul(x, x.T)
    x_square = torch.sum(torch.mul(x, x), dim=-1, keepdim=True)
    return torch.sqrt(x_square + x_inner + x_square.T) #SQRT((x - x.T)^2 = X^2 -2XX.T + X.T^2)

In [53]:
def dense_knn_matrix(x, k=16):
    """Get KNN based on the pairwise distance.
    Args:
        x: (batch_size, num_dims, num_points, 1)
        k: int
    Returns:
        nearest neighbors: (batch_size, num_points ,k) (batch_size, num_points, k)
    """
    with torch.no_grad():
        x = x.transpose(2, 1).squeeze(-1)
        batch_size, n_points, n_dims = x.shape
        _, nn_idx = torch.topk(-pairwise_distance(x.detach()), k=k)
        center_idx = torch.arange(0, n_points, device=x.device).expand(batch_size, k, -1).transpose(2, 1)
    return torch.stack((nn_idx, center_idx), dim=0)

In [145]:
torch.arange(0, 192).expand(1, 4, -1).transpose(2, 1).shape

torch.Size([1, 192, 4])

In [54]:
x = batch

In [55]:
torch.sum(torch.mul(x, x), dim=-1, keepdim=True).shape

torch.Size([1, 196, 1])

In [56]:
(batch @ batch.transpose(1,2)).shape

torch.Size([1, 196, 196])

In [57]:
batch.shape

torch.Size([1, 196, 768])

In [58]:
pairwise_distance(batch).shape

torch.Size([1, 196, 196])

In [116]:
t = torch.tensor(
[[1,2], [2,4]]
)

In [117]:
t.shape

torch.Size([2, 2])

In [118]:
t

tensor([[1, 2],
        [2, 4]])

In [119]:
pairwise_distance(t)

tensor([[0.0000, 2.2361],
        [2.2361, 0.0000]])

In [123]:
(t@t.transpose(0,1))

tensor([[ 5, 10],
        [10, 20]])

In [124]:
t

tensor([[1, 2],
        [2, 4]])

In [125]:
t@t.transpose(0,1)

tensor([[ 5, 10],
        [10, 20]])

In [127]:
-2*torch.matmul(t, t.transpose(0,1))

tensor([[-10, -20],
        [-20, -40]])

In [128]:
torch.matmul(t, t.transpose(0,1))

tensor([[ 5, 10],
        [10, 20]])

---